In [29]:
Nmax =0; Nmin = 0;
S = 1/2; I1 = 3/2; I2 = 1/2; 

BN = 6480.0; DN = 5.544e-3;
gamma = 81.03; 
bBa = 2301; cBa = 75; bFBa = bBa + cBa/3;
#Aparallel = 2376.0; Aperp = 2301.0; bFBa = 1/3 (Aparallel + 2 Aperp); cBa = Aparallel - Aperp;
bF = 63.509; cF = 8.224; bFF = bF + cF/3;
eq0Q = -117;

In [30]:
import numpy as np
from sympy.physics.wigner import wigner_6j,wigner_9j,wigner_3j,clebsch_gordan
from numpy import linalg as LA

def kdel(x,y):
    if x==y:
        return 1
    else:
        return 0
def reduced(x):
    return np.sqrt(x*(x+1)*(2*x+1))
def nreduced(x,y):
    return np.sqrt((2*x+1)*(2*y+1))

def HN(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    return kdel(F,Fp)*kdel(F1,F1p)*kdel(N,Np)*kdel(G,Gp)*(BN*N*(N+1)-DN*N**2*(N+1)**2)

def HNS(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    return (gamma*kdel(F,Fp)*kdel(F1,F1p)*kdel(N,Np)*reduced(N)*reduced(S)*nreduced(G,Gp)*(-1)**(F1+I1+N+S+1+Gp+Gp)
            *wigner_6j(Np,Gp,F1,G,N,1)*wigner_6j(S,Gp,I1,G,S,1))

def HFBa(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    return bFBa/2*kdel(F,Fp)*kdel(F1,F1p)*kdel(N,Np)*kdel(G,Gp)*(G*(G+1)-S*(S+1)-I1*(I1+1))

def HCBa(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    val=0.0
    for J in np.arange(np.abs(N-S),N+S+1):
        for Jp in np.arange(np.abs(Np-S),Np+S+1):
            val += ((-1)**(G+Gp+F1+I1+N+Jp)*nreduced(J,Jp)**2*wigner_6j(F1,G,N,S,J,I1)*wigner_6j(F1p,Gp,Np,S,Jp,I1)*
                    wigner_6j(I1,Jp,F1,J,I1,1)*wigner_9j(J,Jp,1,N,Np,2,S,S,1))
    return (-cBa*np.sqrt(30)/3*kdel(F,Fp)*kdel(F1,F1p)*reduced(I1)*reduced(S)*nreduced(G,Gp)*
            nreduced(N,Np)*wigner_3j(N,2,Np,0,0,0)*val)

def HQ(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    val=0
    for J in np.arange(np.abs(N-S),N+S+1):
        for Jp in np.arange(np.abs(Np-S),Np+S+1):
            val += ((-1)**(G+Gp+Jp+F1+I1+S+Jp)*nreduced(J,Jp)**2*wigner_6j(F1,G,N,S,J,I1)*wigner_6j(F1p,Gp,Np,S,Jp,I1)*
                    wigner_6j(I1,Jp,F1,J,I1,2)*wigner_6j(Np,Jp,S,J,N,2))
    return (eq0Q/4*kdel(F,Fp)*kdel(F1,F1p)*nreduced(G,Gp)*nreduced(N,Np)*
            wigner_3j(N,2,Np,0,0,0)*1/(wigner_3j(I1,2,I1,-I1,0,I1))*val)

def HFF(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    return (bFF*kdel(F,Fp)*kdel(N,Np)*(-1)**(F+I2+N+G+Gp+I1+S)*nreduced(G,Gp)*nreduced(F1,F1p)*reduced(I2)*
            reduced(S)*wigner_6j(I2,F1p,F,F1,I2,1)*wigner_6j(Gp,F1p,N,F1,G,1)*wigner_6j(S,Gp,I1,G,S,1))

def HCF(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    val=0
    for J in np.arange(np.abs(N-S),N+S+1):
        for Jp in np.arange(np.abs(Np-S),Np+S+1):
            val += ((-1)**(G+Gp+F+I2+I1+1+N+J)*nreduced(J,Jp)**2*wigner_6j(F1,G,N,S,J,I1)*wigner_6j(F1p,Gp,Np,S,Jp,I1)
                    *wigner_6j(I2,F1p,F,F1,I2,1)*wigner_6j(Jp,F1p,I1,F1,J,1)*wigner_9j(J,Jp,1,N,Np,2,S,S,1))
    return (-cF*np.sqrt(30)/3*kdel(F,Fp)*reduced(I2)*reduced(S)*nreduced(G,Gp)*
            nreduced(N,Np)*nreduced(F1,F1p)*wigner_3j(N,2,Np,0,0,0)*val)

def H_noB(state):
    return HN(state)+HNS(state)+HFBa(state)+HCBa(state)+HQ(state)+HFF(state)+HCF(state)

In [31]:
#Construct coupled basis
myStates=[[0,0,0,0]];
for n in np.arange(Nmin,Nmax+1,2):
    for g in np.arange(np.abs(I1-S),I1+S+1):
        for f1 in np.arange(np.abs(n-g),n+g+1):
            for f in np.arange(np.abs(f1-I2),f1+I2+1):
                    myStates=np.append(myStates,[[g,n,f1,f]],axis=0)
myStates=np.delete(myStates,0,axis=0)

In [32]:
#Construct the Hamiltonian
#Hailtonian without field
H_0=np.zeros([len(myStates),len(myStates)])
for m in range(len(myStates)):
    for n in range(m+1):
        a=np.append(myStates[m,:],myStates[n,:])
        H_0[m,n]=H_noB(a)
        if m == n:
            H_0[m,n]=H_0[m,n]/2
H_0=H_0+np.transpose(H_0)        

In [33]:
H=H_0
w, v = LA.eig(H)
N=1
w=np.round(w,decimals=5)-(BN*N*(N+1)-DN*N**2*(N+1)**2)
v=np.round(v,decimals=3)
sorted_w=sorted(w)


In [34]:
def matSort(M,index):
    m=len(index)
    C=np.zeros([m,m])
    for i in range(m):
        C[:,i]=M[:,index[i]]
    return C
sort_index = np.argsort(w)
N=1
sorted_w=sorted(w)
sorted_v=matSort(v,sort_index)

In [35]:
#View energy levels with the state composition
for m in range(len(w)):
    n=-m-1
    rr=sorted_v[:,n]
    res = [idx for idx, val in enumerate(rr) if val != 0]
    print(sorted_w[n],end = " ; ")
    for i in res:
        if (i != res[-1]):
            print(rr[i],str(myStates[i,:]),end =" + " )
        else:
            print(rr[i],str(myStates[i,:]))

-11198.915244 ; 1.0 [2.  0.  2.  2.5]
-11240.099794 ; -0.007 [1.  0.  1.  1.5] + 1.0 [2.  0.  2.  1.5]
-15850.915244 ; 1.0 [1.  0.  1.  0.5]
-15875.981024 ; 1.0 [1.  0.  1.  1.5] + 0.007 [2.  0.  2.  1.5]


Shorthand notations:
$$
[a] = \sqrt{2a+1}\\
\{a\} = \sqrt{a(a+1)(2a+1)}
$$

Transformation from Hund's case $b_{\beta S}$ to Hund's case $b_{\beta J}$
$$\scriptsize
|SI_1GNF_1I_2F\rangle = \sum_{J=S+N} [JG] (-1)^{G+S+I_1} 
\left\{\begin{matrix} F_1&G&N\cr S&J&I_1 \end{matrix}\right\}
|NSJI_1F_1I_2F\rangle
$$

State representations
$$ \scriptsize
\psi = |\eta\Lambda; SI_1GNF_1I_2F\rangle; 
\psi' = |\eta\Lambda; SI_1G'N'F'_1I_2F'\rangle
$$

Matrix elements
$$\scriptsize
\langle\psi|H_N|\psi'\rangle = \delta_{FF'}\delta_{F_1F'_1}\delta_{NN'}\delta_{GG'}(B_N N(N+1) - D_N[N(N+1)]^2)
$$

$$\scriptsize
\langle\psi|H_{NS}|\psi'\rangle = \gamma \delta_{FF'}\delta_{F_1F'_1}\delta_{NN'} \{NS\} [GG'] (-1)^{F_1+I_1+N+S+1+G'+G'} \left\{\begin{matrix} N'&G'&F_1\cr G&N&1 \end{matrix}\right\}
\left\{\begin{matrix} S&G'&I_1\cr G&S&1 \end{matrix}\right\}
$$


$$\scriptsize
\langle\psi|H_F^{(Ba)}|\psi'\rangle = \delta_{FF'}\delta_{F_1F'_1}\delta_{NN'}\delta_{GG'} \dfrac{b_F^{(Ba)}}{2}  \left(G(G+1)-S(S+1)-I_1(I_1+1)\right) 
$$

$$\scriptsize
\langle\psi|H_c^{(Ba)}|\psi'\rangle = -\delta_{FF'}\delta_{F_1F'_1} c^{(Ba)}\dfrac{\sqrt{30}}{3}\{I_1S\} [GG'NN']
\sum_{J,J'} (-1)^{G+G'+F_1+I_1+N+J'}[JJ']^2
\left\{\begin{matrix} F_1&G&N\cr S&J&I_1 \end{matrix}\right\}
\left\{\begin{matrix} F_1'&G'&N'\cr S&J'&I_1 \end{matrix}\right\}
\left\{\begin{matrix} I_1&J'&F_1\cr J&I_1&1 \end{matrix}\right\}
\left\{\begin{matrix} J&J'&1\cr N&N'&2\cr S&S&1 \end{matrix}\right\}
$$

$$\scriptsize
\langle\psi|H_Q|\psi'\rangle = \delta_{FF'}\delta_{F_1F'_1} \dfrac{eq_0Q}{4} [GG'NN']
\left(\begin{matrix} N&2&N'\cr 0&0&0 \end{matrix}\right)
\left(\begin{matrix} I_1&2&I_1\cr -I_1&0&I_1 \end{matrix}\right)^{-1}
\sum_{J,J'} (-1)^{G+G'+J'+F_1+I_1+S+J'}[JJ']^2
\left\{\begin{matrix}F_1&G&N\cr S&J&I_1 \end{matrix}\right\}
\left\{\begin{matrix}F_1'&G'&N'\cr S&J'&I_1 \end{matrix}\right\}
\left\{\begin{matrix}I_1&J'&F_1\cr J&I_1&2 \end{matrix}\right\}
\left\{\begin{matrix}N'&J'&S\cr J&N&2 \end{matrix}\right\}
$$

$$\scriptsize
\langle\psi|H_F^{(F)}|\psi'\rangle = \delta_{FF'}\delta_{NN'} (-1)^{F+I_2+N+G+I_1+S}[GG'F_1F_1'] \{SI_2\}
\left\{\begin{matrix} I_2&F_1'&F\cr F_1&I_2&1 \end{matrix}\right\}
\left\{\begin{matrix}G'&F_1'&N\cr F_1&G&1 \end{matrix}\right\}
\left\{\begin{matrix}S&G'&I_1\cr G&S&1 \end{matrix}\right\}
$$

$$\scriptsize
\langle\psi|H_c^{(F)}|\psi'\rangle = -\delta_{FF'} c^{(F)}\dfrac{\sqrt{30}}{3}\{I_2S\} [GG'NN'F_1F_1']
\left(\begin{matrix} N&2&N'\cr 0&0&0 \end{matrix}\right)
\sum_{J,J'} (-1)^{G+G'+F+I_2+I_1+1+N+J}[JJ']^2
\left\{\begin{matrix} F_1&G&N\cr S&J&I_1 \end{matrix}\right\}
\left\{\begin{matrix} F_1'&G'&N'\cr S&J'&I_1 \end{matrix}\right\}
\left\{\begin{matrix} I_2&F_1'&F\cr F_1&I_2&1 \end{matrix}\right\}
\left\{\begin{matrix} J'&F_1'&I_1\cr F_1&J&1 \end{matrix}\right\}
\left\{\begin{matrix} J&J'&1\cr N&N'&2\cr S&S&1 \end{matrix}\right\}
$$